# Create a Graph from a text

This notebook demonstrates how to extract graph from any text using the graph maker

Steps:
- Define an Ontology
- Load a list of example text chunks. We will use the Lord of the Rings summary from this wikipedia page. 
- Create Graph using an Open source model using Groq APIs. 
- Save the graph to Neo4j db
- Visualise



Loading the graph maker functions ->

In [1]:
from knowledge_graph_maker import GraphMaker, Ontology, GroqClient, OpenAIClient
from knowledge_graph_maker import Document

# Define the Ontology. 

The ontology is a pydantic model with the following schema. 

```python
class Ontology(BaseModel):
    label: List[Union[str, Dict]]
    relationships: List[str]
```



In [2]:
from tavily import TavilyClient
client = TavilyClient(api_key="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")
import json

In [3]:
# simple query using tavily's advanced search
response = client.search("Tell me about the Latest Emerging Threats from Iran and the Middle East",
                         search_depth="advanced",
                         max_results=100,
                         include_answer=False,
                         include_images=False,
                         #include_domains=[""],
                         )

response

{'query': 'Tell me about the Latest Emerging Threats from Iran and the Middle East',
 'follow_up_questions': None,
 'answer': None,
 'images': None,
 'results': [{'title': 'Threats in the Middle East - U.S. Department of Defense',
   'url': 'https://www.defense.gov/Spotlights/Threats-in-the-Middle-East/',
   'content': 'Statement of Additional Resources for CENTCOM. Recent attacks are only the latest acts in a 40-year pattern of violence by Iran against the United States and its allies. The facts of the recent ...',
   'score': 0.9488,
   'raw_content': None},
  {'title': "Iran's Attack and the New Escalatory Cycle in the Middle East",
   'url': 'https://www.usip.org/publications/2024/04/irans-attack-and-new-escalatory-cycle-middle-east',
   'content': 'The Middle East is entering a new phase after unprecedented attacks by Israel and Iran during the first two weeks of April. Robin Wright, a senior fellow at USIP and the Woodrow Wilson Center who has covered the region for a half centur

In [4]:
news_docs = []

for doc in response['results']:
    #print(doc)
    title = doc['title']
    content = doc['content']

    compiled_doc = title + " | " + content

    print(compiled_doc)
    news_docs.append(compiled_doc)

    print(" ")


print(len(news_docs))

Threats in the Middle East - U.S. Department of Defense | Statement of Additional Resources for CENTCOM. Recent attacks are only the latest acts in a 40-year pattern of violence by Iran against the United States and its allies. The facts of the recent ...
 
Iran's Attack and the New Escalatory Cycle in the Middle East | The Middle East is entering a new phase after unprecedented attacks by Israel and Iran during the first two weeks of April. Robin Wright, a senior fellow at USIP and the Woodrow Wilson Center who has covered the region for a half century, explores what happened, the strategic implications, the political context and the divided world reaction.
 
US plans more strikes in Middle East against Iran-backed groups | They are the latest blows in a conflict that has spread into the Middle East since Oct. 7, when the Palestinian militant group Hamas stormed Israel from the Gaza Strip, igniting a war that has ...
 
The Iranian Regional Deterrence Strategy in the New Middle East | 

Here is the ontology we will use for the LOTR summaries ->

In [5]:

ontology = Ontology(
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        {"Place": "Geographic location, country, city, or region."},
        "Document",
        {"Organization": "Reprsenting a group of people with common ideals or interests."},
        {"Action": "Activities taken by a Person or Organization."},
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)


## Select a Model

Groq support the following models at present. 

*LLaMA3 8b*
Model ID: llama3-8b-8192

*LLaMA3 70b*
Model ID: llama3-70b-8192

*Mixtral 8x7b*
Model ID: mixtral-8x7b-32768

*Gemma 7b*
Model ID: gemma-7b-it


Selecting a model for this example ->


In [6]:

## Groq models
#model = "mixtral-8x7b-32768"
model ="llama3-8b-8192"
# model = "llama3-70b-8192"
# model="gemma-7b-it"

## Open AI models
#oai_model="gpt-3.5-turbo"

## Use Groq
llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
## OR Use OpenAI
#llm = OpenAIClient(model=oai_model, temperature=0.1, top_p=0.5)


## Create documents out of text chumks. 
Documents is a pydantic model with the following schema 

```python
class Document(BaseModel):
    text: str
    metadata: dict
```

The metadata we add to the document here is copied to every relation that is extracted out of the document. More often than not, the node pairs have multiple relation with each other. The metadata helps add more context to these relations

In this example I am generating a summary of the text chunk, and the timestamp of the run, to be used as metadata. 


In [7]:
import datetime
current_time = str(datetime.datetime.now())


graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary


docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    news_docs
)


## Create Graph
Finally run the Graph Maker to generate graph. 

In [8]:

graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=5 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

Using Model:  llama3-8b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  llama3-8b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  llama3-8b-8192


...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-24 11:02:12 - INFO 
LLM Response:
[
    {
        "node_1": {"label": "Organization", "name": "United States"},
        "node_2": {"label": "Organization", "name": "Israel"},
        "relationship": "The United States is Israel's pre-eminent ally."
    },
    {
        "node_1": {"label": "Organization", "name": "United States"},
        "node_2": {"label": "Place", "name": "Middle East"},
        "relationship": "The United States has military forces in the Middle East."
    },
    {
        "node_1": {"label": "Organization", "name": "Iran"},
        "node_2": {"label": "Organization", "name": "United States"},
        "relationship": "Iran may not target the United States to avoid a direct conflict."
    },
    {
        "node_1": {"label": "Person", "name": "Biden"},
        "node_2": {"label": "Organization", "name": "Iran"},
        "relationship": "Bi

Using Model:  llama3-8b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-24 11:02:18 - INFO 
LLM Response:
[
    {
        "node_1": {"label": "Event", "name": "Iranian attack on Israel"},
        "node_2": {"label": "Person", "name": "Steven A. Cook"},
        "relationship": "Steven A. Cook is an expert who wrote about the Iranian attack on Israel"
    },
    {
        "node_1": {"label": "Event", "name": "Iranian attack on Israel"},
        "node_2": {"label": "Place", "name": "Middle East"},
        "relationship": "The Iranian attack on Israel occurred in the Middle East"
    },
    {
        "node_1": {"label": "Event", "name": "Iranian attack on Israel"},
        "node_2": {"label": "Organization", "name": "U.S."},
        "relationship": "The Iranian attack on Israel presents a challenge for the U.S."
    },
    {
        "node_1": {"label": "Person", "name": "Steven A. Cook"},
        "node_2": {"label": "Organization", 

Using Model:  llama3-8b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-24 11:02:23 - INFO 
LLM Response:
[
    {
        "node_1": {"label": "Person", "name": "Mohammed bin Zayed al-Nahyan"},
        "node_2": {"label": "Person", "name": "Mohammed bin Zayed al-Nahyan"},
        "relationship": "Mohammed bin Zayed al-Nahyan is referring to himself"
    },
    {
        "node_1": {"label": "Person", "name": "Mohammed bin Zayed al-Nahyan"},
        "node_2": {"label": "Organization", "name": "United Arab Emirates"},
        "relationship": "Mohammed bin Zayed al-Nahyan is the ruler of the United Arab Emirates"
    },
    {
        "node_1": {"label": "Person", "name": "Mohammed bin Zayed al-Nahyan"},
        "node_2": {"label": "Event", "name": "Dark Vision of the Middle East's Future"},
        "relationship": "Mohammed bin Zayed al-Nahyan is discussing the Dark Vision of the Middle East's Future"
    }
]


▶︎ GRAPH MAKER LOG - 2

In [ ]:
for edge in graph:
    print(edge.model_dump(exclude=['metadata']), "\n\n")

# Save the Graph to Neo4j 

In [10]:
from knowledge_graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()


aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
a

149

## Georefence Entities

In [ ]:
import requests
import time

# Search Input Terms
key_terms = ['South Korea', 'United States']  ## should come from named_places
layer_0 = "country"
layer_1 = "city"
limit = 1

# Data Collection & Extraction
for term in key_terms:
    print(term)
    # Make a request to the endpoint
    url = f"https://photon.komoot.io/api/?"
    querystring = f"q={term}&layer={layer_0}&layer={layer_1}&limit={limit}"

    response = requests.request("GET", url, params=querystring)

    # Parse the response
    data = response.json()
    print(len(data['features']))

    print(data['features'])
    print(data['features'][0]['geometry']['coordinates'])

    time.sleep(1)